Laboratorio 1 - Regresión lineal 
Integrantes: 
Silvana Echeverry - 202310470
David Mora - 
Carlos Vargas

In [12]:
#importando librerias 

import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import numpy as np
import seaborn as sns

In [13]:
data_train = pd.read_csv('data/datos_entrenamiento_laboratorio1(train_data).csv')
data_test = pd.read_csv('data/datos_validacion_laboratorio1(test_data).csv')

1. Entendimiento de los datos 

Para esta primera etapa, decidimos hacer uso de diferentes funciones para entender como se comportaban nuestros datos, como: 
* Head() -> muestra las primera filas del data frame con las columnas 
* isna().sum() -> nos permitio contar los datos que tenian valores nulos,
* dtypes -> saber los tipos de las variables que tenemos

Despues de hacer esta exploración de los datos, nos dimos cuenta que ciertas variables importantes estaban con valores nulos, o variables importantes estaban con valores que no les correspondian, por ejemplo: 
- edad: object -> y deberia ser int
- market value: object -> y deberia ser int
- dia de partido: object -> fecha 
y con los valores nulos: 
- Edad 48
- Nacionalidad 32
- %_de_regates_exitosos 28.722
- %_de_duelos_aereos_ganados 20.957
- market_value 12.894 (solo esta en el archivo de entrenamiento)
- contract_date 12.916

nota: el pais también deberia cambiarse para dejarlo solamente con su codigo ejm: ARG -> Argentina

Nuestra variable objetivo es predecir el valor de mercado de un jugador (market_value) a partir de sus estadísticas registradas durante partidos. 

Cada fila del data set representa el desempeño de un jugador en un partido especifico (fecha de partido). Nos podemos dar cuenta que hay multiples registros de cada jugador en cada uno de los partidos. 

Además, encontramos que ciertas columnas estan repetidas con datos duplicados como: 
- xAG
- Pasos_Progresivos
- Regates_Exitosos
- Pases_completados

Para hacer una correcta estimación, en esta etapa consideramos que hacen falta variables en cuenta para la estimación, como: 
- partidos_jugados: Cantidad total de partidos registrados por jugador en la temporada
- valor_por_partido: Valor de mercado dividido entre partidos_jugados, ya que segun el partido pueden tener diferente valor
- posicion_general: una variable que categorice la posición del jugador
- años_fincontrato: calcular los años hasta que se le acabe el contrato

Variables mas significativas para nuestro modelo (hasta el momento), consideramos que en esta primera revisión estos son los datos mas relevantes para nuestro modelo 
1. Edad
2. posicion_general (nueva variable)
3. partidos_jugados (nueva variable)
4. Goles
5. xG
6. xA
7. xAG
8. Acciones_que_crean_tiros
9. Pases_al_hueco
10. Pases_en_ultimo_tercio
11. Regates_exitosos
12. Toques_en_area_rival
15. Faltas_cometidas
16. Valla_no_vencida 
17. valor_por_partido (nueva variable)
18. años_fincontrato (nueva variable)
19. Pases_progresivos
20. Pases_intentados
21. Nacionalidad 

In [14]:
' Usamos estas lineas de codigo para explorar los datos '
#data_test.head()
#data_train.head()
#data_train.isna().sum() #verificando datos nulos
#data_test.isna().sum()
#data_test.dtypes
#data_train.dtypes

' Usamos estas lineas de codigo para explorar los datos '

2. Preparación de los datos

Esta etapa lo dividimos en: 
1. Eliminar columnas duplicadas
2. Corregir tipos de datos
3. Tratar valores nulos
4. Normalizar categorías
5. Crear nuevas variables
6. Codificar variables categóricas (one hot encoding)
7. Filtrar registros incompletos
8. Verificar multicolinealidad
9. Separar variables predictoras y variable objetivo

limpiar, fill y luego filtro 

In [15]:
#convertir la fecha de contrato a datetime
data_train['contract_date'] = pd.to_datetime(data_train['contract_date'], errors='coerce')

#calculo el promedio de los contratos de los jugadores para rellenar los nulos, lo hice para revisar cuanto en promedio dura un contrato de un jugador y añadirle esa fecha 
contract_date_mean = data_train['contract_date'].dropna().mean()

data_train['contract_date'] = data_train['contract_date'].fillna(contract_date_mean)

# aca saco la fecha maxima de contrato por jugador para quedarme con la ultima fecha de contrato de cada jugador 
idxl = data_train.groupby('Jugador')['contract_date'].idxmax()

# ahora con esto filtro el data frame para quedarme con la ultima fecha de contrato de cada jugador
data_train = data_train.loc[idxl]


/var/folders/0g/xm2gc5sx67sds29_nv2g25n00000gn/T/ipykernel_53906/727014576.py:2: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data_train['contract_date'] = pd.to_datetime(data_train['contract_date'], errors='coerce')


In [16]:

# 1. Asegurar formato datetime
data_train['Dia_partido'] = pd.to_datetime(data_train['Dia_partido'], errors='coerce')

# 2. Crear variable con el número de partidos jugados por cada jugador
partidos_jugados = data_train.dropna(subset=['Dia_partido']).groupby('Jugador').size().reset_index(name='Partidos_jugados')

# 3. Unir esa variable al DataFrame original
data_train = data_train.merge(partidos_jugados, on='Jugador', how='left')

# 4. Obtener el índice del último partido por jugador (usando el nuevo data_train con Partidos_jugados)
idx_fecha = data_train.dropna(subset=['Dia_partido']).groupby('Jugador')['Dia_partido'].idxmax()

# 5. Filtrar el DataFrame para dejar solo la última fila por jugador
data_train = data_train.loc[idx_fecha]

data_train

,Jugador,Nacionalidad,Posicion,Edad,Dia_partido,Goles,Tiros Totales,xG,npxG,xAG,...,Perdida_balon,Pases_recibidos,Pases_progresivos_recibidos,Faltas_cometidas,Centros,Duelos_aereos_ganados,%_de_duelos_aereos_ganados,market_value,contract_date,Partidos_jugados
0,Aaron Ciammaglichella,it ITA,CM,19-212,2024-08-25,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,NaN,€700k,2026-06-30 00:00:00.000000000,1.0
1,Aaron Cresswell,eng ENG,LB,34-253,2024-08-24,0,0,0.0,0.0,0.0,...,0,1,0,0,0,1,100.0,NaN,2028-04-05 09:41:01.780937216,1.0
2,Aaron Malouda,fr FRA,FW,19-056,2025-01-25,0,0,0.0,0.0,0.0,...,1,1,0,0,0,0,NaN,NaN,2028-04-05 09:41:01.780937216,1.0
3,Aaron Martin,es ESP,"WB,LM",27-117,2024-08-17,0,0,0.0,0.0,0.0,...,0,17,3,1,6,0,0.0,€6.50m,2026-06-30 00:00:00.000000000,1.0
4,Aaron Ramsdale,eng ENG,GK,26-109,2024-08-31,0,0,0.0,0.0,0.0,...,0,28,0,0,0,0,NaN,€10.00m,2028-06-30 00:00:00.000000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2237,Ørjan Nyland,no NOR,GK,33-341,2024-08-16,0,0,0.0,0.0,0.0,...,0,33,0,0,0,0,NaN,€900k,2026-06-30 00:00:00.000000000,1.0
2238,Þorir Johann Helgason,is ISL,"LW,LM",24-058,2024-11-25,0,1,0.0,0.0,0.0,...,0,3,2,1,0,0,NaN,€3.00m,2026-06-30 00:00:00.000000000,1.0
2239,Đorđe Petrovic,rs SRB,GK,24-343,2024-09-15,0,0,0.0,0.0,0.0,...,0,43,0,0,0,0,NaN,NaN,2028-04-05 09:41:01.780937216,1.0
2240,Łukasz Fabianski,pl POL,GK,39-135,2024-08-31,0,0,0.0,0.0,0.0,...,0,2,0,0,0,0,NaN,NaN,2028-04-05 09:41:01.780937216,1.0


In [17]:
# convertir Edad a string para trabajar con texto para que la division sea mas facil
data_train["Edad"] = data_train["Edad"].astype(str)

# eliminar lo que está después del guion, que no nos sirve
data_train["Edad"] = data_train["Edad"].str.split('-').str[0]

# reemplazar 'nan' (como texto) por np.nan
data_train["Edad"] = data_train["Edad"].replace("nan", np.nan)

# convertir a número (forzar errores a NaN)
data_train["Edad"] = pd.to_numeric(data_train["Edad"], errors='coerce')

# rellenar los valores nulos con 0
data_train["Edad"] = data_train["Edad"].fillna(0)

# obtener el índice de la fila con la edad máxima por jugador
idxe = data_train.groupby('Jugador')["Edad"].idxmax()

# filtrar el DataFrame para dejar solo esa fila por jugador
data_train = data_train.loc[idxe]

In [18]:
# Convertir a string por si hay valores no string
data_train["Nacionalidad"] = data_train["Nacionalidad"].astype(str)

# Extraer las últimas 3 letras (el código del país)
data_train["Nacionalidad"] = data_train["Nacionalidad"].str[-3:]

# Reemplazar nulos si los hay (opcional: puedes usar un valor como 'UNK' o dejar np.nan)
data_train["Nacionalidad"] = data_train["Nacionalidad"].replace("nan", np.nan)
data_train["Nacionalidad"] = data_train["Nacionalidad"].fillna("UNK")  # 'UNK' = unknown

In [19]:
data_train['market_value_f'] = data_train['market_value'].str.replace('-', '0')
data_train['market_value_f'] = data_train['market_value_f'].str.replace('error', '0')
data_train['market_value_f'] = data_train['market_value_f'].str.replace('€', '')
data_train['market_value_ultimodigito'] = data_train['market_value_f'].str[-1] #ultimo digito
data_train['market_value_ultimodigito'] = data_train['market_value_ultimodigito'].map({'m': 10000000, 'k': 1000})
data_train['market_value_f'] = data_train['market_value_f'].str.replace('k', '').str.replace('m', '').astype(float)
data_train['market_value_f'] = data_train['market_value_f'] * data_train['market_value_ultimodigito']


data_train = data_train.dropna(subset=['market_value']) #aca eliminamos las que estan vacian y no vale la pena usarlas 

#data_train[data_train['market_value_f'].isna()] #verficaa con 1 y 0 si esta nula, filtra las filas de ese valor en null 

In [20]:
# 1) Listas de posiciones (en MAYÚSCULAS)
porteros = {"GK"}

defensas = {"CB", "LB", "RB", "WB", "RWB", "LWB"}  # centrales, laterales y carrileros

centrocampistas = {
    "DM", "CM", "AM",  # mediocentro defensivo, centrocampista, mediapunta
    "LM", "RM",        # volantes/extremos de medio campo
    # variantes comunes opcionales:
    "LDM","RDM","LCM","RCM","LAM","RAM"
}

delanteros = {"FW", "ST", "CF", "LW", "RW"}  # delantero, punta, segundo punta, extremos

# 2) Función de mapeo (maneja posiciones múltiples tipo "DM,CM")
def map_posicion_general(pos):
    if pd.isna(pos):
        return np.nan
    etiquetas = [p.strip().upper() for p in str(pos).split(",") if p.strip()]

    # Prioridad si hay múltiples etiquetas: Portero > Defensa > Centrocampista > Delantero
    if any(p in porteros for p in etiquetas):
        return "Portero"
    if any(p in defensas for p in etiquetas):
        return "Defensa"
    if any(p in centrocampistas for p in etiquetas):
        return "Centrocampista"
    if any(p in delanteros for p in etiquetas):
        return "Delantero"


#crear la nueva variable 
data_train["posicion_general"] = data_train["Posicion"].apply(map_posicion_general)

'''preguntar'''
# (Opcional) convertir a categórica con orden útil
data_train["posicion_general"] = pd.Categorical(
    data_train["posicion_general"],
    categories=["Portero","Defensa","Centrocampista","Delantero","Otra"],
    ordered=False
)

data_train



,Jugador,Nacionalidad,Posicion,Edad,Dia_partido,Goles,Tiros Totales,xG,npxG,xAG,...,Faltas_cometidas,Centros,Duelos_aereos_ganados,%_de_duelos_aereos_ganados,market_value,contract_date,Partidos_jugados,market_value_f,market_value_ultimodigito,posicion_general
0,Aaron Ciammaglichella,ITA,CM,19.0,2024-08-25,0,0,0.0,0.0,0.0,...,0,0,0,NaN,€700k,2026-06-30,1.0,700000.0,1000.0,Centrocampista
3,Aaron Martin,ESP,"WB,LM",27.0,2024-08-17,0,0,0.0,0.0,0.0,...,1,6,0,0.0,€6.50m,2026-06-30,1.0,65000000.0,10000000.0,Defensa
4,Aaron Ramsdale,ENG,GK,26.0,2024-08-31,0,0,0.0,0.0,0.0,...,0,0,0,NaN,€10.00m,2028-06-30,1.0,100000000.0,10000000.0,Portero
5,Aaron WanBissaka,ENG,RB,26.0,2024-08-24,0,0,0.0,0.0,0.0,...,0,0,0,NaN,€24.00m,2031-06-30,1.0,240000000.0,10000000.0,Defensa
6,Abakar Sylla,CIV,CB,21.0,2024-08-18,0,0,0.0,0.0,0.0,...,3,0,1,100.0,€10.00m,2028-06-30,1.0,100000000.0,10000000.0,Defensa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231,Zeki Celik,TUR,RB,27.0,2024-08-18,0,1,0.0,0.0,0.0,...,1,3,1,100.0,€6.00m,2026-06-30,1.0,60000000.0,10000000.0,Defensa
2234,Zion Suzuki,JPN,GK,21.0,2024-08-17,0,0,0.0,0.0,0.0,...,1,0,1,100.0,€20.00m,2029-06-30,1.0,200000000.0,10000000.0,Portero
2235,Zito Luvumbo,ANG,FW,22.0,2024-08-18,0,1,0.1,0.1,0.1,...,5,1,1,50.0,€5.00m,2028-06-30,1.0,50000000.0,10000000.0,Delantero
2237,Ørjan Nyland,NOR,GK,33.0,2024-08-16,0,0,0.0,0.0,0.0,...,0,0,0,NaN,€900k,2026-06-30,1.0,900000.0,1000.0,Portero


In [21]:
# === DEDUP por CONTENIDO (misma info celda a celda) ===
mask_keep = ~data_train.T.duplicated(keep='first')   # True=conservar, False=duplicada
keep_idx = np.where(mask_keep)[0]
drop_idx = np.where(~mask_keep)[0]

cols_to_drop = data_train.columns[drop_idx].tolist()

print("Columnas a ELIMINAR por ser duplicadas de alguna anterior (índice, nombre):")
for i in drop_idx:
    print(f"{i:>3d}  {data_train.columns[i]}")
print("\nTotal eliminadas:", len(drop_idx))
print("Total resultantes:", len(keep_idx))

# Eliminar por posición (índice)
data_train = data_train.iloc[:, keep_idx].copy()

# Chequeo rápido
print("\nPrimeras columnas tras limpieza:")
print(data_train.columns[:20].tolist())

data_train


Columnas a ELIMINAR por ser duplicadas de alguna anterior (índice, nombre):
 10  xAG.1
 13  Pases_intentados.1
 14  Pases_intentados.2
 16  Pases_progresivos.1
 18  Regates_exitosos.1
 21  xAG.2
 22  xAG.3
 30  Pases_completados.1
 31  Pases_completados.2
 45  Centros

Total eliminadas: 10
Total resultantes: 44

Primeras columnas tras limpieza:
['Jugador', 'Nacionalidad', 'Posicion', 'Edad', 'Dia_partido', 'Goles', 'Tiros Totales', 'xG', 'npxG', 'xAG', 'Acciones_que_crean_tiros', 'Pases_intentados', 'Pases_progresivos', 'Regates_exitosos', 'Pases_medios_completados', 'Pases_largos_completados', 'xA', 'Pases_en_ultimo_tercio', 'Pases_balon_vivo', 'Pases_balon_muerto']


,Jugador,Nacionalidad,Posicion,Edad,Dia_partido,Goles,Tiros Totales,xG,npxG,xAG,...,Pases_progresivos_recibidos,Faltas_cometidas,Duelos_aereos_ganados,%_de_duelos_aereos_ganados,market_value,contract_date,Partidos_jugados,market_value_f,market_value_ultimodigito,posicion_general
0,Aaron Ciammaglichella,ITA,CM,19.0,2024-08-25,0,0,0.0,0.0,0.0,...,0,0,0,NaN,€700k,2026-06-30,1.0,700000.0,1000.0,Centrocampista
3,Aaron Martin,ESP,"WB,LM",27.0,2024-08-17,0,0,0.0,0.0,0.0,...,3,1,0,0.0,€6.50m,2026-06-30,1.0,65000000.0,10000000.0,Defensa
4,Aaron Ramsdale,ENG,GK,26.0,2024-08-31,0,0,0.0,0.0,0.0,...,0,0,0,NaN,€10.00m,2028-06-30,1.0,100000000.0,10000000.0,Portero
5,Aaron WanBissaka,ENG,RB,26.0,2024-08-24,0,0,0.0,0.0,0.0,...,0,0,0,NaN,€24.00m,2031-06-30,1.0,240000000.0,10000000.0,Defensa
6,Abakar Sylla,CIV,CB,21.0,2024-08-18,0,0,0.0,0.0,0.0,...,0,3,1,100.0,€10.00m,2028-06-30,1.0,100000000.0,10000000.0,Defensa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231,Zeki Celik,TUR,RB,27.0,2024-08-18,0,1,0.0,0.0,0.0,...,4,1,1,100.0,€6.00m,2026-06-30,1.0,60000000.0,10000000.0,Defensa
2234,Zion Suzuki,JPN,GK,21.0,2024-08-17,0,0,0.0,0.0,0.0,...,0,1,1,100.0,€20.00m,2029-06-30,1.0,200000000.0,10000000.0,Portero
2235,Zito Luvumbo,ANG,FW,22.0,2024-08-18,0,1,0.1,0.1,0.1,...,5,5,1,50.0,€5.00m,2028-06-30,1.0,50000000.0,10000000.0,Delantero
2237,Ørjan Nyland,NOR,GK,33.0,2024-08-16,0,0,0.0,0.0,0.0,...,0,0,0,NaN,€900k,2026-06-30,1.0,900000.0,1000.0,Portero
